<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Очистка-данных" data-toc-modified-id="Очистка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Очистка данных</a></span></li><li><span><a href="#Создание-токенов-и-их-лемматизация" data-toc-modified-id="Создание-токенов-и-их-лемматизация-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Создание токенов и их лемматизация</a></span></li><li><span><a href="#Удаление-длинных-и-коротких-комментариев" data-toc-modified-id="Удаление-длинных-и-коротких-комментариев-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Удаление длинных и коротких комментариев</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Трейн,Тест,-Валид" data-toc-modified-id="Трейн,Тест,-Валид-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Трейн,Тест, Валид</a></span></li><li><span><a href="#Feature&amp;Target" data-toc-modified-id="Feature&amp;Target-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Feature&amp;Target</a></span></li><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#RidgeClassifier" data-toc-modified-id="RidgeClassifier-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>RidgeClassifier</a></span></li><li><span><a href="#SVC" data-toc-modified-id="SVC-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>SVC</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
mport numpy as np
import pandas as pd
import torch
import transformers
from tqdm import notebook

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score,precision_score, recall_score, precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.pipeline import make_pipeline, Pipeline
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
import lightgbm as lgb
from sklearn.linear_model import RidgeClassifier

import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import nltk

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

from IPython.display import display
from datetime import datetime 


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Vladimir\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vladimir\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vladimir\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
comments = pd.read_csv(r'C:\Users\Vladimir\Documents\Job\Course 13 text\jupyter\toxic_comments.csv')

In [3]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Почти 160 тысяч комментариев, пропусков нет, типы и названия столбцов нас устраивают.

In [4]:
def check_balance(df):
    print(df['toxic'].value_counts(normalize=True))
check_balance(comments)

0    0.898321
1    0.101679
Name: toxic, dtype: float64


Видим дисбаланс классов 90%-10%, нужно будет с этим что то делать. Будем менять баланс перед обучением моделей и только на трейне.

### Очистка данных
Заметим наличие служебных символов, попробуем очистить их.

In [5]:
#import string
#string.punctuation
def standardize_text(df, text_field):
    df['text_orig'] = df[text_field]
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\']", " ")#\n\'\`\"\_
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.lower()
    return df
#display(standardize_text(comments, "text"))
df = comments.copy()

### Создание токенов и их лемматизация

In [6]:
tokenizer = RegexpTokenizer(r'\w+')#W+ means that either a word character (A-Za-z0-9_) or a dash (-) can go there.
df["tokens"] = df["text"].apply(tokenizer.tokenize)

In [7]:
wn = nltk.WordNetLemmatizer()
stop_words = stopwords.words('english')
stop_words.remove('not')
snowball = SnowballStemmer(language="english")
ps = nltk.stem.porter.PorterStemmer()

In [8]:
text_list = []
for token in df['tokens']:
    text = [word for word in token if word not in stop_words]
    text = [snowball.stem(word) for word in text]
    text = ' '.join(text)
    text_list.append(text)
    del text
df2 = pd.concat([df,pd.Series(text_list)],axis=1)


In [16]:
df_copy = df2.copy()

In [13]:
df2.columns = ['text', 'toxic', 'tokens','lemmas']

### Удаление длинных и коротких комментариев

In [14]:
count_list = []
for token in df2['tokens']:
    count_list.append(len(token))
count = pd.DataFrame(count_list,columns=['count'],index=df.index)
df2 = pd.concat([df2,count],axis=1)
#print(df.sort_values(by='count',ascending=False))

In [15]:
df2 = df2[(~(df2['count']>=1000) | (df2['count']<=1))]
df2

,text,toxic,tokens,lemmas,count
0,Explanation\nWhy the edits made under my usern...,0,"[Explanation, Why, the, edits, made, under, my...",explan whi edit made usernam hardcor metallica...,50
1,D'aww! He matches this background colour I'm s...,0,"[D, aww, He, matches, this, background, colour...",d aww he match background colour i seem stuck ...,20
2,"Hey man, I'm really not trying to edit war. It...",0,"[Hey, man, I, m, really, not, trying, to, edit...",hey man i realli not tri edit war it guy const...,44
3,"""\nMore\nI can't make any real suggestions on ...",0,"[More, I, can, t, make, any, real, suggestions...",more i make real suggest improv i wonder secti...,114
4,"You, sir, are my hero. Any chance you remember...",0,"[You, sir, are, my, hero, Any, chance, you, re...",you sir hero ani chanc rememb page,14
...,...,...,...,...,...
159566,""":::::And for the second time of asking, when ...",0,"[And, for, the, second, time, of, asking, when...",and second time ask view complet contradict co...,46
159567,You should be ashamed of yourself \n\nThat is ...,0,"[You, should, be, ashamed, of, yourself, That,...",you asham that horribl thing put talk page 128...,21
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,"[Spitzer, Umm, theres, no, actual, article, fo...",spitzer umm there actual articl prostitut ring...,11
159569,And it looks like it was actually you who put ...,0,"[And, it, looks, like, it, was, actually, you,...",and look like actual put speedi first version ...,25


In [17]:
df2['toxic'].value_counts()

0    143338
1     16187
Name: toxic, dtype: int64

## Обучение

### Трейн,Тест, Валид

In [18]:
model,test = train_test_split(df2[['lemmas','toxic']],test_size=0.2,shuffle=True, random_state=890,stratify=df2['toxic'])
train,valid = train_test_split(model,test_size=0.25,shuffle=True, random_state=890,stratify=model['toxic'])

### Feature&Target

In [34]:
def feature_target(df):
    X =  df['lemmas']
    
    y = df['toxic']
    return X,y
X,y = feature_target(df2)
X_model,y_model = feature_target(model)
X_train,y_train = feature_target(train.sample(10000))
X_valid,y_valid = feature_target(valid.sample(10000))
X_test,y_test   = feature_target(test)

X_test = X_test.values.astype('U')
X_model = X_model.values.astype('U')

### LogisticRegression

Запустим гридсеч на трейне и подберем гиперпараметры модели. Прогнав несколько раз гридсерч пришло пониманию что необходимо варировать коэфициентом регуляризации, т.к. на гиперпараметрах
<br> ``{'fit_intercept' : True, penalty': 'l2', 'solver': 'liblinear'}``
<br>  Модель дает стабильно высокий результат. Причем значение С в районе 5-15

In [22]:
# Создадим словарь с парамет
grid_params = {
    # Key = step name from pipeline + __ + hyperparameter, value = tuple of possible values
    #'logreg__penalty': ('l1', 'l2'),
    'logreg__C': np.logspace(start=0.7,stop=1.5,num=5),
    #'logreg__fit_intercept': (True,False),
    #'logreg__solver':('liblinear','saga')
}

#Создадим последовательность для пайалайна
estimators = [('TfIdf',TfidfVectorizer(stop_words=stop_words)),
              ('logreg',LogisticRegression(penalty = 'l2',solver='liblinear',class_weight='balanced', random_state=42)   )
             ]
pipe = Pipeline(estimators)

#GridSearch
grid = GridSearchCV(pipe, n_jobs=-1, param_grid=grid_params,cv=5, scoring='f1')
grid.fit(X_model, y_model) 
mean_scores = np.array(grid.cv_results_['mean_test_score'])
print("Значение метрики F1 на кросс-валидации равно", mean_scores.mean())
#print(pipe)

Значение метрики F1 на кросс-валидации равно 0.749162742876173


Нам удалось побить baseline на трэйне. Теперь трансформируем представление гиперпараметров, чтобы модель на тесте их поняла.

In [23]:
# преобазуем словарь с параметрами модели
def transform_params(best_params):
    _dict={}
    for param in  best_params:
        _dict[param.replace('logreg__','')]=best_params[param]
    return _dict
best_params = transform_params(grid.best_params_)    
best_params    

{'C': 7.943282347242813}

In [59]:
start_time = datetime.now() 
estimators = [('TfIdf',TfidfVectorizer(stop_words=stop_words)),
              ('logreg',LogisticRegression(C=8,penalty = 'l2',solver='liblinear',class_weight='balanced', random_state=42)   )
             ]
pipe2 = Pipeline(estimators)
pipe2.fit(X_model, y_model)
y_pred = pipe2.predict(X_test)
print("Значение метрики F1 на трейне равно",f1_score(y_test,y_pred))

time_calc = (datetime.now() - start_time).seconds
print('Время расчета пайплайна  {} секунд '.format(time_calc))

Значение метрики F1 на трейне равно 0.763517359134889
Время расчета пайплайна  7 секунд 


На тесте нам  удалось побить бэйзлайн. Результат даже оказался лучше, чем на кросс валидации. Да и скорость данного алгоритма самая быстрая, по сравнению с другими.

### RidgeClassifier

Попробуем линейную модель с регуляризацией. Исходя из экспериментов с логистической регрессией мы выберем тип регуляризации l2.

In [38]:
#time
start_time = datetime.now() 

# Создадим словарь с парамет
grid_params = {
    # Key = step name from pipeline + __ + hyperparameter, value = tuple of possible values
    #'ridge__penalty': ('l1', 'l2'),
    'ridge__alpha': [1,2,3,4],
   # 'ridge__fit_intercept': (True,False),
    #'ridge__solver':('sag','saga')
}

#Создадим последовательность для пайалайна
estimators = [('TfIdf',TfidfVectorizer(stop_words=stop_words)),
              ('ridge',RidgeClassifier(solver='sag',class_weight='balanced', random_state=42)   )
             ]
pipe = Pipeline(estimators)

#GridSearch
grid = GridSearchCV(pipe, n_jobs=-1, param_grid=grid_params,cv=5, scoring='f1')
grid.fit(X_model, y_model)
mean_scores = np.array(grid.cv_results_['mean_test_score'])
print("Значение метрики F1 на кросс-валидации равно", mean_scores.mean())
print(pipe)


#time
time_calc = (datetime.now() - start_time).seconds
print('Время расчета пайплайна  {} секунд '.format(time_calc))

Значение метрики F1 на кросс-валидации равно 0.2752911283245886
Pipeline(steps=[('TfIdf',
                 TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('ridge',
                 RidgeClassifier(class_weight='balanced', random_state=42))])
Время расчета пайплайна  28 секунд 


In [39]:
grid.best_params_

{'ridge__alpha': 4}

In [44]:
estimators = [('TfIdf',TfidfVectorizer(stop_words=stop_words,min_df=30)),
              ('logreg',RidgeClassifier(alpha = 4,solver='sag',class_weight='balanced', random_state=42)   )
             ]
pipe = Pipeline(estimators)

pipe.fit(X_model, y_model)
y_pred = pipe.predict(X_test)
print('Test',f1_score(y_test,y_pred))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:555: UserWarning: "sag" solver requires many iterations to fit an intercept with sparse inputs. Either set the solver to "auto" or "sparse_cg", or set a low "tol" and a high "max_iter" (especially if inputs are not standardized).
  warnings.warn(


Test 0.6639869363145334


Линейная регрессия  подходит для решения данной задачи результат F1 в районе 0.66. Что тоже оч неплохо, но все же не дотягивает до бэйзлайна

### SVC

In [57]:
#time
start_time = datetime.now() 

# Создадим словарь с парамет
grid_params = {
    #'SVC__kernel': ('linear', 'poly', 'rbf', 'sigmoid', 'precomputed'),
    #'SVC__degree': [3,4,5,7],
    'SVC__C': [5,7,9,11],
    #'SVC__gamma':('auto','scale')
}

#Создадим последовательность для пайалайна
estimators = [('TfIdf',TfidfVectorizer(stop_words=stop_words)),
              ('SVC',SVC(random_state=890))
             ]
pipe = Pipeline(estimators)

#GridSearch
grid = GridSearchCV(pipe, n_jobs=-1, param_grid=grid_params,cv=5, scoring='f1')
grid.fit(X_test, y_test)
mean_scores = np.array(grid.cv_results_['mean_test_score'])
print("Значение метрики F1 на кросс-валидации равно", round(mean_scores.mean(),2))
print(pipe)
#time
time_calc = (datetime.now() - start_time).seconds
print('Время расчета пайплайна  {} секунд '.format(time_calc))

Значение метрики F1 на кросс-валидации равно 0.34
Pipeline(steps=[('TfIdf',
                 TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('SVC', SVC(random_state=890))])
Время расчета пайплайна  486 секунд 


In [58]:
estimators = [('TfIdf',TfidfVectorizer(stop_words=stop_words)),
              ('SVC',SVC(kernel = 'linear',C=8,random_state=890))   
             ]
pipe = Pipeline(estimators)

pipe.fit(X_model, y_model)
y_pred = pipe.predict(X_test)
print('Test',f1_score(y_test,y_pred))

Test 0.7721995094031071


Очень неплохо на тесте показал себя метод опорных векторов 0.77 по метрике F1.

## Выводы

`Очистка данных`
<br>На вход нам поступил датасэт со 160 тысячами комментариев, из которых 10% было размечено, как токсичные, а 90% как не токсичные. 
<br> С помощью регулярных выражений мы убрали служебные символы, кавычки, ссылки.
Провели токенизацию и лемматизацию текстов из корпуса, перевели тексты в Юникод.

Была попытка сделать downsampling, но это не привело к повышению точности модели. Поэтому было принято решение обучать на всем датасэте, используюя кросс-валидация и параметр взвешивания классов ``class_weight='balanced'``

При проверке моделей на дефолтных параметрах было выяснено, что Логистическая регрессия дает лучший результат. Прогнав модель по сетке гиперпараметров мы увидели что лучший результат по метрике F1 (0.75) выходит с регуляризацией при параметрах {'fit_intercept' : True, penalty': 'l2', 'solver': 'liblinear'}. Но так же необходим была более тонкая настройка с подбором коэфициента регуляризации C. На тесте получили 0.76 

Пробовали и линейные модели c Регуляризацией L2 (Ridge). Она показала очень  результат на метрике F1. 0.66

Метод опорных веркторов (SVC) Так же не плохо справился с задачей, 0.77 на кросс валидации.

У данной задачи есть потенциал по обработке текста и тюнингу гиперпараметров модели. Но в рамках данного проекта мы решили не развивать данные направления.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны